In [7]:
# pip install eeglib

# eeglib

[Documentation](https://eeglib.readthedocs.io/en/latest/index.html)

[Github](https://github.com/Xiul109/eeglib/blob/master/readme.md)

In [8]:
from eeglib.helpers import CSVHelper
import pandas as pd
import numpy as np

In [10]:
csv_path="./eeglib-master/Examples/fake_EEG_signal.csv"

## Let's first checkout the csv file the common way, without the use of the package
(read as a pandas DataFrame obect not a eeglib Helper object)

In [33]:
pd.read_csv(csv_path)

,C1,C2,C3,C4,C5
0,4.400000,5.400000,4.400000,3.400000,4.400000
1,2.830176,6.830176,2.830176,5.830176,1.830176
2,0.591038,-0.408962,4.591038,5.591038,5.591038
3,-1.288087,-1.288087,-2.288087,-2.288087,0.711913
4,0.788888,0.788888,-1.211112,1.788888,-3.211112
...,...,...,...,...,...
1019,-1.860936,1.139064,2.139064,-2.860936,0.139064
1020,-1.494807,0.505193,-1.494807,-1.494807,0.505193
1021,1.289493,1.289493,2.289493,2.289493,-3.710507
1022,-1.975354,-0.975354,1.024646,1.024646,-2.975354


## Basic Example

Now we'll read it in with the package as an eeglib Helper object

In [4]:
helper= CSVHelper(csv_path)

# PFD: Petrosian Fractal Dimension
for eeg in helper:
    print(eeg.PFD())

[1.03089233 1.03229887 1.03181488 1.03123267 1.03069761]


## Using Windows

In [29]:
helper= CSVHelper(csv_path,windowSize=256)

i = 1
for eeg in helper:
    print(i,":", eeg.PFD())
    i = i+1

1 : [1.03922468 1.03897773 1.03971798 1.03674636 1.03873059]
2 : [1.03848326 1.04168343 1.04094783 1.04168343 1.03699509]
3 : [1.03996434 1.04045647 1.03996434 1.03774006 1.03947143]
4 : [1.03749194 1.04045647 1.03897773 1.0402105  1.03873059]


Now the function has been called 4 times, this is because of the data has a length of 1024 samples and the window selected has a size of 256, so the windows contained in the data are 1024/256=4.

In [30]:
helper= CSVHelper(csv_path,windowSize=128)

i = 1
for eeg in helper:
    print(i,":", eeg.PFD())
    i = i+1

1 : [1.04536601 1.04365174 1.04250474 1.04307866 1.04135438]
2 : [1.04422399 1.04536601 1.04764022 1.04077793 1.04650474]
3 : [1.04020062 1.04707289 1.04650474 1.04933738 1.04307866]
4 : [1.04707289 1.04877246 1.04707289 1.04650474 1.04135438]
5 : [1.04933738 1.04650474 1.04764022 1.04536601 1.04707289]
6 : [1.04250474 1.04536601 1.04365174 1.04020062 1.04365174]
7 : [1.04536601 1.04536601 1.04764022 1.04536601 1.04707289]
8 : [1.03962247 1.04650474 1.04135438 1.04593579 1.04135438]


## Using iterators to create overlapping windows

In [12]:
helper= CSVHelper(csv_path,windowSize=256)

for eeg in helper[::128]:
    print(eeg.PFD())

[1.03922468 1.03897773 1.03971798 1.03674636 1.03873059]
[1.03674636 1.04045647 1.04143842 1.03971798 1.03897773]
[1.03848326 1.04168343 1.04094783 1.04168343 1.03699509]
[1.04217287 1.04192825 1.04119322 1.0402105  1.03873059]
[1.03996434 1.04045647 1.03996434 1.03774006 1.03947143]
[1.03848326 1.03996434 1.03996434 1.03749194 1.03996434]
[1.03749194 1.04045647 1.03897773 1.0402105  1.03873059]


## Preprocessing Functions:
1. Bandpass filtering
2. Z-Scores normalization
3. Independent Component Analysis
 
These preprocessings can be applied at the load of the data by the Helpers:

In [15]:
helper = CSVHelper(csv_path,lowpass=30, highpass=1, normalize=True, ICA=True)

## Using Wrappers
A Wrapper is an object that envelops a helper and simplifies the proccess of computing features that can be later be used, for example, in machine learning algorithms. The next example shows an example of how wrappers can be used:

In [17]:
from eeglib import wrapper, helpers

helper = helpers.CSVHelper(csv_path, windowSize=128)

wrap = wrapper.Wrapper(helper)

wrap.addFeature.HFD()
wrap.addFeature.DFT()
wrap.addFeature.synchronizationLikelihood()

features=wrap.getAllFeatures()

So, the scheme to follow with wrappers is the next:

1. Create the Helper object.
2. Create the wrapper object.
3. Select the desired features to compute. They can be parameterized by adding the parameters just behind the name.
4. Call the method "getAllFeatures()" in order to compute every feature from every window at once or iterate over the Wrapper object for obtaining the features of each window. They are returned as a pandas.DataFrame or a pandas.Series.


In [23]:
features.keys()

Index(['HFD(){}_0', 'HFD(){}_1', 'HFD(){}_2', 'HFD(){}_3', 'HFD(){}_4',
       'DFT(){}_0_0', 'DFT(){}_0_1', 'DFT(){}_0_2', 'DFT(){}_0_3',
       'DFT(){}_0_4',
       ...
       'synchronizationLikelihood(){}_0', 'synchronizationLikelihood(){}_1',
       'synchronizationLikelihood(){}_2', 'synchronizationLikelihood(){}_3',
       'synchronizationLikelihood(){}_4', 'synchronizationLikelihood(){}_5',
       'synchronizationLikelihood(){}_6', 'synchronizationLikelihood(){}_7',
       'synchronizationLikelihood(){}_8', 'synchronizationLikelihood(){}_9'],
      dtype='object', length=655)

In [24]:
features

,HFD(){}_0,HFD(){}_1,HFD(){}_2,HFD(){}_3,HFD(){}_4,DFT(){}_0_0,DFT(){}_0_1,DFT(){}_0_2,DFT(){}_0_3,DFT(){}_0_4,...,synchronizationLikelihood(){}_0,synchronizationLikelihood(){}_1,synchronizationLikelihood(){}_2,synchronizationLikelihood(){}_3,synchronizationLikelihood(){}_4,synchronizationLikelihood(){}_5,synchronizationLikelihood(){}_6,synchronizationLikelihood(){}_7,synchronizationLikelihood(){}_8,synchronizationLikelihood(){}_9
window_id,,,,,,,,,,,,,,,,,,,,,
0,2.075391,2.071034,2.121366,2.113046,2.122448,12.670161+ 0.000000j,29.297981-51.296463j,-3.366936+38.722112j,14.247080+62.601966j,11.291793-50.912786j,...,0.094828,0.054598,0.143678,0.129310,0.066327,0.127551,0.114796,0.113757,0.074074,0.104816
1,2.165376,2.127756,2.142201,2.185759,2.142667,-19.670161+ 0.000000j,-36.266134-83.734034j,11.226158-35.866990j,-8.206202-46.517039j,17.750423+73.805636j,...,0.120275,0.079038,0.092784,0.137457,0.090129,0.188841,0.163090,0.056856,0.113712,0.243976
2,2.088513,2.126656,2.119470,2.101255,2.122609,15.670161+ 0.000000j,12.588284-47.935057j,-21.228263+ 2.555405j,27.300341+86.701638j,9.379021-64.625159j,...,0.037594,0.120301,0.157895,0.120301,0.020672,0.085271,0.087855,0.169591,0.084795,0.077830
3,2.132532,2.147984,2.123692,2.176121,2.180053,-20.670161+ 0.000000j,-12.277473-91.838156j,10.305358-34.444358j,7.268723-71.943727j,3.448646+92.749064j,...,0.293706,0.104895,0.132867,0.115385,0.099723,0.116343,0.141274,0.148734,0.079114,0.086826
4,2.119542,2.149438,2.103158,2.072021,2.122049,-1.329839+ 0.000000j,10.579100-50.510962j,14.001457-21.587785j,1.061275+55.241769j,27.067234-33.798876j,...,0.089474,0.102632,0.107895,0.144737,0.086262,0.073482,0.067093,0.081928,0.069880,0.111111
5,2.152988,2.141881,2.150179,2.176188,2.149479,-2.670161+ 0.000000j,-7.941925-54.517145j,-5.134929-43.750696j,-16.458681-85.035971j,-24.136815+59.337621j,...,0.085714,0.180000,0.145714,0.102857,0.090909,0.071429,0.107143,0.127596,0.083086,0.148276
6,2.129761,2.104530,2.093110,2.110664,2.092894,-0.329839+ 0.000000j,9.350791-45.787999j,25.987804+32.941247j,-2.240485+59.370250j,29.194686-55.326200j,...,0.172749,0.121655,0.150852,0.184915,0.118605,0.125581,0.172093,0.112045,0.238095,0.183066
7,2.176519,2.149823,2.146592,2.109823,2.123788,-39.670161+ 0.000000j,-20.237357-72.695304j,-32.611933-28.229149j,17.368302-70.278109j,-25.493546+69.083622j,...,0.179331,0.136778,0.085106,0.191489,0.075067,0.104558,0.142091,0.102113,0.186620,0.121864
